In [133]:
import os, math
import numpy as np
from numba import cuda
import matplotlib.pyplot as plt
from tqdm import tqdm

### Data Loading

In [ ]:
def make_uint32(b):
    return (b[0] << 24) | (b[1] << 16) | (b[2] << 8) | b[3]

def read_labels(filename):
    with open(filename, 'rb') as f:
        _ = f.read(4)
        n = make_uint32(f.read(4))
        labels = np.frombuffer(f.read(n), dtype=np.uint8)
    return labels

def read_images(filename):
    with open(filename, 'rb') as f:
        _ = f.read(4)
        n = make_uint32(f.read(4))
        rows = make_uint32(f.read(4))
        cols = make_uint32(f.read(4))
        images = np.frombuffer(f.read(n * rows * cols), dtype=np.uint8).reshape(n, rows*cols)
    return images

### CUDA Kernels

**Kernel index calculation**

```python
tx = cuda.threadIdx.x
ty = cuda.threadIdx.y
bx = cuda.blockIdx.x
by = cuda.blockIdx.y
bw = cuda.blockDim.x
bh = cuda.blockDim.y
x = tx + bx * bw
y = ty + by * bh
```

OR :

```python
x, y = cuda.grid(2)
```


In [ ]:
@cuda.jit
def matmul_kernel(A, B, C):
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        s = 0.0
        for k in range(A.shape[1]):
            s += A[i, k] * B[k, j]
        C[i, j] = s

@cuda.jit
def add_bias_kernel(Z, b):
    i, j = cuda.grid(2)
    if i < Z.shape[0] and j < Z.shape[1]:
        Z[i, j] += b[i, 0]

@cuda.jit
def sigmoid_kernel(A):
    i, j = cuda.grid(2)
    if i < A.shape[0] and j < A.shape[1]:
        A[i, j] = 1.0 / (1.0 + math.exp(-A[i, j]))

### Parallel Functions

In [ ]:
def get_grid(shape, tpb):
    return (math.ceil(shape[0] / tpb[0]), math.ceil(shape[1] / tpb[1]))

def gpu_matmul(A, B):
    A_gpu = cuda.to_device(A)
    B_gpu = cuda.to_device(B)
    C = np.empty((A.shape[0], B.shape[1]), dtype=np.float64)
    C_gpu = cuda.to_device(C)
    dev = cuda.get_current_device()
    tpb = (int(math.sqrt(dev.MAX_THREADS_PER_BLOCK)), int(math.sqrt(dev.MAX_THREADS_PER_BLOCK)))
    grid = get_grid(C.shape, tpb)
    matmul_kernel[grid, tpb](A_gpu, B_gpu, C_gpu)
    C_gpu.copy_to_host(C)
    return C

def gpu_add_bias(Z, b):
    Z_gpu = cuda.to_device(Z)
    b_gpu = cuda.to_device(b)
    dev = cuda.get_current_device()
    tpb = (int(math.sqrt(dev.MAX_THREADS_PER_BLOCK)), int(math.sqrt(dev.MAX_THREADS_PER_BLOCK)))
    grid = get_grid(Z.shape, tpb)
    add_bias_kernel[grid, tpb](Z_gpu, b_gpu)
    Z_gpu.copy_to_host(Z)
    return Z

def gpu_sigmoid(A):
    A_gpu = cuda.to_device(A)
    dev = cuda.get_current_device()
    tpb = (int(math.sqrt(dev.MAX_THREADS_PER_BLOCK)), int(math.sqrt(dev.MAX_THREADS_PER_BLOCK)))
    grid = get_grid(A.shape, tpb)
    sigmoid_kernel[grid, tpb](A_gpu)
    A_gpu.copy_to_host(A)
    return A


### Neural Network

In [ ]:
def sigmoid(x):
    return 1.0/(1.0+np.exp(-x))
def dsigmoid(x):
    s = sigmoid(x)
    return s*(1-s)

class ANN:
    def __init__(self, sizes, alpha, minibatch_size):
        self.alpha = alpha
        self.minibatch = minibatch_size
        self.L = len(sizes)-1
        self.W = [np.random.randn(sizes[i+1], sizes[i]) / np.sqrt(sizes[i])
                  for i in range(self.L)]
        self.b = [np.zeros((sizes[i+1],1)) for i in range(self.L)]
        
    def forward(self, x):
        a = x.copy()
        self.z_list= []
        self.a_list = [a]
        for l in range(self.L):
            z = gpu_matmul(self.W[l], a)
            z = gpu_add_bias(z, self.b[l])
            self.z_list.append(z)
            a = gpu_sigmoid(z)
            self.a_list.append(a)
        return a

    def backward(self, x, y):
        # Compute gradients via backpropagation (on CPU)
        L = self.L
        delta = (self.a_list[-1] - y) * dsigmoid(self.z_list[-1])
        gradW = [None]*L
        gradb = [None]*L
        gradW[L-1] = np.dot(delta, self.a_list[L-1].T) / self.minibatch
        gradb[L-1] = np.sum(delta, axis=1, keepdims=True) / self.minibatch
        for l in range(L-2, -1, -1):
            delta = np.dot(self.W[l+1].T, delta) * dsigmoid(self.z_list[l])
            gradW[l] = np.dot(delta, self.a_list[l].T) / self.minibatch
            gradb[l] = np.sum(delta, axis=1, keepdims=True) / self.minibatch
        # Update weights and biases
        for l in range(L):
            self.W[l] -= self.alpha * gradW[l]
            self.b[l] -= self.alpha * gradb[l]

    def evaluate(self, X, y):
        n = X.shape[1]
        output = self.forward(X)
        predictions = np.argmax(output, axis=0)
        return np.sum(predictions == y)


### Main Execution

In [137]:
# Training settings and main loop
DATA_PATH = "DATA"
train_img = read_images(os.path.join(DATA_PATH, "train-images.idx3-ubyte"))
train_label = read_labels(os.path.join(DATA_PATH, "train-labels.idx1-ubyte"))
test_img = read_images(os.path.join(DATA_PATH, "t10k-images.idx3-ubyte"))
test_label = read_labels(os.path.join(DATA_PATH, "t10k-labels.idx1-ubyte"))

alpha = 0.05
minibatch = 16
sizes = [784, 30, 10]
net = ANN(sizes, alpha, minibatch)
NEPOCHS = 5

def one_hot(indices, n_classes):
    m = np.zeros((n_classes, len(indices)))
    m[indices, np.arange(len(indices))] = 1.0
    return m


  0%|          | 0/3750 [00:00<?, ?it/s]

100%|██████████| 3750/3750 [00:31<00:00, 118.31it/s]


Epoch 0 - Acc: 80.80%, CE: 1.3444


100%|██████████| 3750/3750 [00:52<00:00, 70.78it/s]


Epoch 1 - Acc: 84.40%, CE: 0.7748


...

Epoch 2 - Acc: 84.00%, CE: 0.6675


100%|██████████| 3750/3750 [00:52<00:00, 71.22it/s]


Epoch 3 - Acc: 83.80%, CE: 0.6162


100%|██████████| 3750/3750 [00:55<00:00, 67.87it/s]

Epoch 4 - Acc: 85.40%, CE: 0.5782


In [ ]:
N_train = train_img.shape[0]
for epoch in range(NEPOCHS):
    indices = np.arange(N_train)
    np.random.shuffle(indices)
    ce_total = 0.0
    n_batches = 0
    for i in tqdm(range(0, N_train - minibatch + 1, minibatch)):
        batch_idx = indices[i:i+minibatch]
        x = (train_img[batch_idx].T.astype(np.float64)) / 255.0
        y = one_hot(train_label[batch_idx], 10)
        output = net.forward(x)
        # Cross-entropy loss (CPU)
        eps = 1e-12
        output_clip = np.clip(output, eps, 1-eps)
        ce = -np.sum(y * np.log(output_clip)) / minibatch
        ce_total += ce
        net.backward(x, y)
        n_batches += 1
    acc = net.evaluate((test_img[:1000].T.astype(np.float64))/255.0, test_label[:1000])
    print(f"Epoch {epoch} - Acc: {100.0*acc/1000:.2f}%, CE: {ce_total/n_batches:.4f}")

In [138]:
# Final evaluation on test set (using first 1000 examples for speed)
acc = net.evaluate((test_img[:1000].T.astype(np.float64))/255.0, test_label[:1000])
print(f"Final Test Accuracy: {100.0*acc/1000:.2f}%")


Final Test Accuracy: 85.40%
